# Antimikrobiyal Direnç Tahmin Modeli
Ülke bazında 2025-2030 direnç tahmini (Polynomial Regression)

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")
print("✓ Kütüphaneler yüklendi")

✓ Kütüphaneler yüklendi


In [28]:
# Veriyi yükle ve hazırla
ecoli_df = pd.read_csv('ecoli.csv')

# Sadece direnç yüzdesi verilerini filtrele
resistance_df = ecoli_df[ecoli_df['Indicator'] == 'R - resistant isolates, percentage  '].copy()
simplified_df = resistance_df[['Time', 'RegionName', 'Population', 'NumValue']].copy()
simplified_df.columns = ['Year', 'Country', 'Bacteria_Antibiotic', 'Resistance_Percentage']

# Bakteri ve antibiyotik bilgisini ayır
simplified_df[['Bacteria', 'Antibiotic']] = simplified_df['Bacteria_Antibiotic'].str.split('|', expand=True)
simplified_df = simplified_df[['Year', 'Country', 'Bacteria', 'Antibiotic', 'Resistance_Percentage']]
simplified_df['Resistance_Percentage'] = pd.to_numeric(simplified_df['Resistance_Percentage'], errors='coerce')

# Yıl ve ülke bazında ortalama
final_df = simplified_df.groupby(['Year', 'Country'])['Resistance_Percentage'].mean().reset_index()
final_df.columns = ['Year', 'Country', 'Avg_Resistance_%']
final_df = final_df.dropna()

print(f"✓ Veri hazır: {len(final_df)} satır, {final_df['Country'].nunique()} ülke")
final_df.head(10)

✓ Veri hazır: 717 satır, 30 ülke


,Year,Country,Avg_Resistance_%
0,2000,Austria,17.801047
1,2000,Belgium,20.852359
2,2000,Bulgaria,40.196078
3,2000,Czechia,4.271845
4,2000,Denmark,0.199601
6,2000,Finland,1.362398
7,2000,Germany,9.197878
8,2000,Greece,21.006396
9,2000,Iceland,2.500000
10,2000,Ireland,37.964775


In [ ]:
# Her ülke için ayrı polynomial regression modeli
country_models = {}
country_trends = {}

for country in final_df['Country'].unique():
    country_data = final_df[final_df['Country'] == country].sort_values('Year')
    
    if len(country_data) < 3:
        continue
    
    X = country_data[['Year']].values
    y = country_data['Avg_Resistance_%'].values
    
    # Polynomial Regression (degree=2)
    model = Pipeline([
        ('poly', PolynomialFeatures(degree=2)),
        ('linear', LinearRegression())
    ])
    model.fit(X, y)
    
    country_models[country] = model
    country_trends[country] = {
        'years': country_data['Year'].values,
        'values': country_data['Avg_Resistance_%'].values
    }

print(f"✓ {len(country_models)} ülke için model oluşturuldu")

Sadeleştirilmiş Veri:
     Year     Country            Bacteria       Antibiotic  \
734  2012    Bulgaria  Acinetobacter spp.  Aminoglycosides   
735  2012      Cyprus  Acinetobacter spp.  Aminoglycosides   
736  2012     Germany  Acinetobacter spp.  Aminoglycosides   
737  2012     Denmark  Acinetobacter spp.  Aminoglycosides   
738  2012      Greece  Acinetobacter spp.  Aminoglycosides   
739  2012      France  Acinetobacter spp.  Aminoglycosides   
740  2012     Hungary  Acinetobacter spp.  Aminoglycosides   
741  2012     Iceland  Acinetobacter spp.  Aminoglycosides   
742  2012       Italy  Acinetobacter spp.  Aminoglycosides   
743  2012  Luxembourg  Acinetobacter spp.  Aminoglycosides   

    Resistance_Percentage  
734          58.461538460  
735          52.173913040  
736           5.882352940  
737          10.389610380  
738          78.119935170  
739          12.949640280  
740          68.796068790  
741                     -  
742          83.333333330  
743            

In [ ]:
# Gelecek tahminleri (2025-2030)
future_predictions = []

for country, model in country_models.items():
    for year in range(2025, 2031):
        pred = model.predict([[year]])[0]
        pred = max(0, pred)
        future_predictions.append({
            'Year': year,
            'Country': country,
            'Predicted_Resistance_%': round(pred, 2)
        })

future_df = pd.DataFrame(future_predictions)
future_df.to_csv('future_predictions_2025_2030.csv', index=False)

print(f"✓ {len(future_df)} tahmin yapıldı")
print("\n📊 Örnek tahminler:")
future_df.head(15)

In [6]:
# Eksik değerleri temizle (- işaretlerini NaN yap)
simplified_df['Resistance_Percentage'] = pd.to_numeric(simplified_df['Resistance_Percentage'], errors='coerce')

# Yıl ve ülke bazında ortalama direnç
country_year_avg = simplified_df.groupby(['Year', 'Country'])['Resistance_Percentage'].mean().reset_index()
country_year_avg.columns = ['Year', 'Country', 'Avg_Resistance_%']
country_year_avg['Avg_Resistance_%'] = country_year_avg['Avg_Resistance_%'].round(2)

print("EN BASIT FORMAT - Yıl ve Ülke Bazında Ortalama Direnç:")
print(country_year_avg.head(20))
print(f"\nToplam satır: {len(country_year_avg)}")

EN BASIT FORMAT - Yıl ve Ülke Bazında Ortalama Direnç:
    Year      Country  Avg_Resistance_%
0   2000      Austria             17.80
1   2000      Belgium             20.85
2   2000     Bulgaria             40.20
3   2000      Czechia              4.27
4   2000      Denmark              0.20
5   2000      Estonia               NaN
6   2000      Finland              1.36
7   2000      Germany              9.20
8   2000       Greece             21.01
9   2000      Iceland              2.50
10  2000      Ireland             37.96
11  2000        Italy             44.26
12  2000   Luxembourg             17.91
13  2000        Malta             35.53
14  2000  Netherlands              4.83
15  2000       Norway              3.61
16  2000     Portugal             25.17
17  2000     Slovenia             21.05
18  2000        Spain             29.40
19  2000       Sweden              0.61

Toplam satır: 722


In [ ]:
# Trend görselleştirmesi
top_countries = ['Greece', 'Romania', 'Bulgaria', 'Italy', 'Poland']

plt.figure(figsize=(14, 8))

for country in top_countries:
    if country not in country_models:
        continue
    
    hist_years = country_trends[country]['years']
    hist_values = country_trends[country]['values']
    
    future_data = future_df[future_df['Country'] == country]
    future_years = future_data['Year'].values
    future_values = future_data['Predicted_Resistance_%'].values
    
    plt.plot(hist_years, hist_values, 'o-', linewidth=2, markersize=6, 
             label=f'{country} (Gerçek)', alpha=0.7)
    plt.plot(future_years, future_values, 's--', linewidth=2, markersize=7, alpha=0.9)

plt.axvline(x=2024.5, color='red', linestyle=':', linewidth=2, alpha=0.5)
plt.title('Antimikrobiyal Direnç Trendleri (2000-2030)', fontsize=15, fontweight='bold')
plt.xlabel('Yıl', fontsize=13)
plt.ylabel('Ortalama Direnç (%)', fontsize=13)
plt.legend(loc='best', fontsize=10, ncol=2)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

✓ 30 ülke için ayrı model oluşturuldu

🔮 Örnek Tahminler (Ülke Bazında Model):

📍 Greece:
   2025: 47.37%
   2026: 47.33%
   2027: 47.20%
   2028: 47.00%
   2029: 46.71%
   2030: 46.35%

📍 Romania:
   2025: 47.60%
   2026: 47.34%
   2027: 46.97%
   2028: 46.49%
   2029: 45.91%
   2030: 45.21%

📍 Bulgaria:
   2025: 50.50%
   2026: 51.85%
   2027: 53.23%
   2028: 54.63%
   2029: 56.05%
   2030: 57.49%

📍 Italy:
   2025: 39.10%
   2026: 39.69%
   2027: 40.29%
   2028: 40.90%
   2029: 41.52%
   2030: 42.15%

📍 Poland:
   2025: 37.38%
   2026: 37.03%
   2027: 36.58%
   2028: 36.03%
   2029: 35.38%
   2030: 34.63%



In [ ]:
# 2030 yılı risk analizi
risk_2030 = future_df[future_df['Year'] == 2030].sort_values('Predicted_Resistance_%', ascending=False).head(10)

plt.figure(figsize=(12, 6))
bars = plt.barh(risk_2030['Country'], risk_2030['Predicted_Resistance_%'], 
                color=plt.cm.RdYlGn_r(risk_2030['Predicted_Resistance_%']/100))
plt.xlabel('Tahmin Edilen Direnç (%)', fontsize=12)
plt.title('En Yüksek Risk - 2030 Tahmini', fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3)

for i, (idx, row) in enumerate(risk_2030.iterrows()):
    plt.text(row['Predicted_Resistance_%'] + 0.5, i, f"{row['Predicted_Resistance_%']:.1f}%", 
             va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n⚠️ 2030 RİSK SIRLAMASI:")
for idx, row in risk_2030.iterrows():
    print(f"   {row['Country']}: {row['Predicted_Resistance_%']:.2f}%")

📊 Ülke Bazında Model Performansı (En İyi 15):

       Country  MAE    R²  Data_Points
      Slovakia 1.78 0.952           19
       Croatia 1.58 0.947           23
        Greece 1.75 0.933           25
     Lithuania 1.55 0.911           19
        Cyprus 1.74 0.901           22
       Denmark 1.47 0.822           25
        Poland 2.97 0.747           24
       Hungary 2.27 0.736           24
        Latvia 2.73 0.733           21
      Bulgaria 3.23 0.720           25
       Ireland 2.85 0.700           25
United Kingdom 2.28 0.647           20
       Romania 4.60 0.598           23
       Czechia 2.55 0.590           25
       Austria 0.95 0.543           25

📈 Ortalama Performans:
   Ortalama MAE: 2.14
   Ortalama R²: 0.525

✅ Ülke bazında tahminler kaydedildi: future_predictions_country_based_2025_2030.csv


In [ ]:
# Detaylı tahminler - örnek ülkeler
example_countries = ['Greece', 'Romania', 'Bulgaria']

print("🔮 2025-2030 Detaylı Tahminler:\n")
for country in example_countries:
    if country in country_models:
        country_future = future_df[future_df['Country'] == country]
        print(f"📍 {country}:")
        for _, row in country_future.iterrows():
            print(f"   {row['Year']}: {row['Predicted_Resistance_%']:.2f}%")
        print()

---
## ℹ️ Model Bilgileri

**Model Tipi:** Ülke Bazında Polynomial Regression (degree=2)

**Özellikler:**
- ✅ Her ülke için ayrı trend modeli
- ✅ Gerçekçi artış/azalış tahminleri
- ✅ 2025-2030 arası 6 yıllık tahmin

**Çıktı Dosyası:** `future_predictions_2025_2030.csv`